In [ ]:
%pip install librosa

In [16]:
import librosa
import librosa.display

import IPython.display as ipd
import matplotlib.pyplot as plt

In [24]:
aud_file = "../sampleAudio/tgRunIt.wav"
deb = "../sampleAudio/debussy.wav"
duke = "../sampleAudio/duke.wav"
redhot = "../sampleAudio/redhot.wav"

In [ ]:
ipd.Audio(aud_file)

In [25]:
tg, sr = librosa.load(aud_file)
# signals, sampling rates
deb_sig, deb_sr = librosa.load(deb)
duke_sig, duke_sr = librosa.load(duke)
redhot_sig, redhot_sr = librosa.load(redhot)

In [ ]:
tg.shape

In [ ]:
# duration 1 sample
sam_dur = 1/sr
sam_dur

In [ ]:
duration = sam_dur * len(tg)
# total duration of signal
duration/60 # min 

In [ ]:
# vis wave form

plt.figure(figsize=(45,17))
# plt.subplot(3,1,1)
librosa.display.waveshow(tg, sr=sr)
plt.ylim((-1,1))
plt.title("tg run it")


In [ ]:
# to loud look at other samples now 
plt.figure(figsize=(15, 17))

plt.subplot(3, 1, 1)
librosa.display.waveshow(deb_sig, sr=deb_sr)
plt.ylim((-1, 1))
plt.title("Debusy")

plt.subplot(3, 1, 2)
librosa.display.waveshow(redhot_sig, sr=redhot_sr)
plt.ylim((-1, 1))
plt.title("RHCP")

plt.subplot(3, 1, 3)
librosa.display.waveshow(duke_sig, sr=duke_sr)
plt.ylim((-1, 1))
plt.title("Duke Ellington")

plt.show()

$\text{max env} = max(signal)$

In [36]:
# audio envelop
import numpy as np

FRAME_SIZE = 1024
HOP_LEN = 512

def ampl_env(signal, frame_size, hop_len):
    # overlapping frames
    amp_env = []
    for i in range(0, len(signal), hop_len):
        amp_env.append(max(signal[i:i+frame_size]))
    
    return np.array(amp_env)

def better_ample_env(signal, frame_size, hop_len):
    return np.array([max(signal[i:i+frame_size]) for i in range(0,signal.size, hop_len)])

In [ ]:
ampl_deb = ampl_env(deb_sig, FRAME_SIZE, HOP_LEN)
len(ampl_deb)

In [37]:
red_ae = better_ample_env(redhot_sig, FRAME_SIZE, HOP_LEN)
deb_ae = better_ample_env(deb_sig, FRAME_SIZE, HOP_LEN)
duke_ae = better_ample_env(duke_sig, FRAME_SIZE, HOP_LEN)


In [ ]:
plt.figure(figsize=(15, 17))
frames = range(0,deb_ae.size)
t = librosa.frames_to_time(frames, hop_length=HOP_LEN)

plt.subplot(3, 1, 1)
librosa.display.waveshow(deb_sig, sr=deb_sr)
plt.plot(t,deb_ae)
plt.ylim((-1, 1))
plt.title("Debusy")

plt.subplot(3, 1, 2)
librosa.display.waveshow(redhot_sig, sr=redhot_sr)
plt.plot(t,red_ae)
plt.ylim((-1, 1))
plt.title("RHCP")

plt.subplot(3, 1, 3)
librosa.display.waveshow(duke_sig, sr=duke_sr)
plt.plot(t,duke_ae)
plt.ylim((-1, 1))
plt.title("Duke Ellington")

plt.show()

# rmse and z crossing

Rmse- avergage energy or intensity of the signal over time(like avg loudness/power)
$ \\ RMSE_t = \sqrt{\frac{1}{K} \sum_{k=t*k}^{(t+1)*K-1} s(k)^2} \\$
square ampltiude values, mean of all squared amp on a envelop/packet, sqrt mean

In [ ]:
deb_rbse = librosa.feature.rms(y=deb_sig, frame_length=FRAME_SIZE, hop_length=HOP_LEN)
deb_rbse.shape

In [51]:
# create our own
def rms(signal, frame_len, hop_len):
    rms = []
    for i in range(0, len(signal), hop_len):
        rms_curr = np.sqrt(np.sum(signal[i:i+frame_len]**2)/frame_len)
        rms.append(rms_curr)
    return np.array(rms)


In [55]:
deb_rbse2 = rms(deb_sig, FRAME_SIZE, HOP_LEN)
red_rbse2 = rms(redhot_sig, FRAME_SIZE, HOP_LEN)
duke_rbse2 = rms(duke_sig, FRAME_SIZE, HOP_LEN)

red_rbse = librosa.feature.rms(y=redhot_sig, frame_length=FRAME_SIZE, hop_length=HOP_LEN)
duke_rbse = librosa.feature.rms(y=duke_sig, frame_length=FRAME_SIZE, hop_length=HOP_LEN)


In [ ]:
(deb_rbse2 == deb_rbse).all()

In [ ]:
print(deb_rbse)
print("*************************")
print(deb_rbse2)

In [ ]:
plt.figure(figsize=(15, 17))
frames = range(0,deb_ae.size)
t = librosa.frames_to_time(frames, hop_length=HOP_LEN)

plt.subplot(3, 1, 1)
librosa.display.waveshow(deb_sig, sr=deb_sr)

plt.ylim((-1, 1))
plt.plot(t, deb_rbse2, color="r")
# plt.plot(t, deb_rbse.flatten(), color="b")
plt.title("Debusy")

plt.subplot(3, 1, 2)
librosa.display.waveshow(redhot_sig, sr=redhot_sr)
plt.ylim((-1, 1))
plt.plot(t, red_rbse2, color="r")
# plt.plot(t, red_rbse.flatten(), color="b")
plt.title("RHCP")

plt.subplot(3, 1, 3)
librosa.display.waveshow(duke_sig, sr=duke_sr)
plt.ylim((-1, 1))
plt.plot(t, duke_rbse2, color="r")
# plt.plot(t, duke_rbse.flatten(), color="b")
plt.title("Duke Ellington")

plt.show()

zero crossing rate
amount of times the sin wave crosses zero, thats it

$$
zcr = \frac{1}{T-1}\sum_{t=1}^{T} |s(t) - s(t-1)|
$$

In [65]:
deb_zc = librosa.feature.zero_crossing_rate(deb_sig, frame_length=FRAME_SIZE)[0]
duke_zc = librosa.feature.zero_crossing_rate(duke_sig, frame_length=FRAME_SIZE)[0]
red_zc = librosa.feature.zero_crossing_rate(redhot_sig, frame_length=FRAME_SIZE)[0]

In [ ]:
plt.figure(figsize=(15, 17))
plt.plot(t, deb_zc*FRAME_SIZE, color="g")
plt.plot(t, duke_zc*FRAME_SIZE, color="y")
plt.plot(t, red_zc*FRAME_SIZE, color="r") # non normalized
plt.show()

Fourier Transform